# Protostars

## 01- Exploratory Data Analysis
### I. Metadata

This notebook explores the metadata downloaded from the [Keck archive](http://www2.keck.hawaii.edu/koa/public/koa.php).  I queried for the coordinated of S68N, and then output the Keck Archive table results to a csv file `metadata/sci_ns_29923.tbl`.  

Michael Gully-Santiago  
February 21, 2017


Let's see how many observations of this target there are, and what the observation properties were.

In [1]:
import pandas as pd
import astropy

In [2]:
from astropy.io import ascii

In [3]:
sci_tbl = ascii.read('../data/metadata/sci_ns_29923.tbl')

In [4]:
sci_df = sci_tbl.to_pandas()

In [5]:
sci_df.head(2)

,koaid,instrument,targname,object,imagetyp,frameno,ra,dec,date_obs,ut,...,filehand,semid,propmin,detmode,dispers,disppos,echlpos,filname,slitname,prop
0,NC.20110611.47928.fits,NIRSPEC,Serp 68N/SMM9,HIP 91748 A0,object,62.0,277.45083,1.27850,2011-06-11,13:18:48.57,...,/koadata7/NIRSPEC/20110611/lev0/scam/NC.201106...,2011a_n091ns,18.0,scam,NaN,35.61,179.7,NIRSPEC-7,42x0.760,f
1,NC.20110611.47949.fits,NIRSPEC,Serp 68N/SMM9,HIP 91748 A0,object,63.0,277.45222,1.27989,2011-06-11,13:19:09.37,...,/koadata7/NIRSPEC/20110611/lev0/scam/NC.201106...,2011a_n091ns,18.0,scam,NaN,35.61,179.7,NIRSPEC-7,42x0.760,f


In [6]:
sci_df.columns

Index(['koaid', 'instrument', 'targname', 'object', 'imagetyp', 'frameno',
       'ra', 'dec', 'date_obs', 'ut', 'elaptime', 'waveblue', 'wavered',
       'progid', 'proginst', 'progpi', 'progtitl', 'sig2nois', 'slitwidt',
       'filehand', 'semid', 'propmin', 'detmode', 'dispers', 'disppos',
       'echlpos', 'filname', 'slitname', 'prop'],
      dtype='object')

It's a sparse table, just look for unique values.

In [7]:
cols = list(set(sci_df.columns)-{'koaid', 'filehand', 'sig2nois', 'frameno', 'dec', 'ut', 'ra'})
for col in cols:
    print("{:<20}".format(col), sci_df[col].unique())

disppos              [ 35.61  35.76  35.45]
wavered              [ 2.63]
echlpos              [ 179.7    61.89]
imagetyp             ['object']
dispers              [nan 'high' 'low']
instrument           ['NIRSPEC']
date_obs             ['2011-06-11' '2013-06-19' '2014-06-18' '2014-06-19' '2003-06-20']
progtitl             [ 'Spectroscopy of the Nearest, Youngest, Lowest Mass Candidate Planetary Mass Object'
 'Studying the photosphere of a Class 0 protostar'
 'The Formation of Substellar Objects']
prop                 ['f']
object               ['HIP 91748 A0' 'Ser S68N Class 0' 'Ser S68N' 'Oph BD 387 NE'
 'HIP 91748 tel STD' 'HIP 91748 A0 tel' 'Kr Lamp' 'HIP 92486 A0 tel'
 'EC 38' 'Ser 68N / SMM9' 'Ser S68B' 'Ser S68N CLass 0' 'Ser S68 N Class 0']
progid               ['N091NS' 'N062NS' 'N090NS' 'N8NS']
waveblue             [ 1.84]
slitwidt             [   nan  0.576  0.76 ]
detmode              ['scam' 'spec']
slitname             ['42x0.760' '0.576x12']
filname              ['NIRSP

So the wavelength range is from 1.84 $\mu$m to 2.63 $\mu$m, and the resolution is "low".  
The low resolution mode corresponds to $R\sim2264$ for an $0.42''$ slit according to the [NIRSPEC website](http://www2.keck.hawaii.edu/koa/public/nspec/nirspec_data_description.html).  
These observations used a $0.76''$ slit, so the $R$ should be:

In [8]:
est_R = 2264.0/0.76*0.42
print("R ~ {:.1f}".format(est_R))

R ~ 1251.2


What is the on-source integration time?

In [9]:
for s, group in sci_df.groupby('object'):
    print("{:<20}".format(s), "{:<10.2f}".format(group.elaptime.sum()/3600.0), 
          group.slitname.unique(), group.dispers.unique(), group['filname'].unique())

EC 38                0.50       ['0.576x12'] ['high'] ['NIRSPEC-7']
HIP 91748 A0         0.00       ['42x0.760'] [nan] ['NIRSPEC-7']
HIP 91748 A0 tel     0.00       ['42x0.760'] [nan] ['NIRSPEC-7']
HIP 91748 tel STD    0.01       ['42x0.760'] [nan] ['NIRSPEC-7']
HIP 92486 A0 tel     0.02       ['42x0.760'] [nan] ['NIRSPEC-7']
Kr Lamp              0.01       ['42x0.760'] [nan] ['NIRSPEC-7']
Oph BD 387 NE        0.02       ['42x0.760'] [nan] ['NIRSPEC-7']
Ser 68N / SMM9       0.57       ['42x0.760'] ['low'] ['NIRSPEC-7']
Ser S68 N Class 0    0.40       ['42x0.760'] ['low'] ['NIRSPEC-7']
Ser S68B             0.27       ['42x0.760'] ['low'] ['NIRSPEC-7']
Ser S68N             2.87       ['42x0.760'] [nan 'low'] ['NIRSPEC-7']
Ser S68N CLass 0     0.53       ['42x0.760'] ['low'] ['NIRSPEC-7']
Ser S68N Class 0     3.09       ['42x0.760'] [nan 'low'] ['NIRSPEC-7']


Looks like a whole night on Keck in total.  Not bad!

According to the [Keck NIRSPEC page](http://www2.keck.hawaii.edu/inst/nirspec/Specifications.html), The 'NIRSPEC-7' filter has a wavelength range: 1.839-2.630 $\mu$m, consistent with the range stated above.